In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# installing necessary applications for 
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!sudo add-apt-repository "deb http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/"
!sudo apt-get update
!sudo apt-get -y install cuda

# Install JoeyNMT
! git clone https://github.com/may-/joeynmt.git
! cd joeynmt; pip3 install .
# Install Pytorch with GPU support v1.9.0
! pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

--2021-09-22 04:53:10--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu1804.pin’

cuda-ubuntu1804.pin 100%[===================>]     190  --.-KB/s    in 0s      

2021-09-22 04:53:10 (13.8 MB/s) - ‘cuda-ubuntu1804.pin’ saved [190/190]

Executing: /tmp/apt-key-gpghome.WCCOyBB2ik/gpg.1.sh --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
gpg: requesting key from 'https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub'
gpg: key F60F4B3D7FA2AF80: "cudatools <cudatools@nvidia.com>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Er

In [ ]:
import os
from os import path
tag = "pmi_nhs_da_bt"
tokenized_data = "/content/drive/MyDrive/DA/updated_data_tok_files"
src_lang = "hi"
tgt_lang = "en"
bpe_operations = 5000
bpe_file = "len_full.bpe.codes." + str(bpe_operations)
bpe_folder = src_lang + "_" + tgt_lang + "_" + tag + "_" + str(bpe_operations)
bpe_data = "/content/drive/MyDrive/DA/" + bpe_folder + "/"

os.environ["src_lang"] = src_lang
os.environ["tgt_lang"] = tgt_lang
os.environ["tokenized_data"] = tokenized_data
os.environ["bpe_file"] = bpe_file
os.environ["bpe_operations"] = str(bpe_operations)
os.environ["jnmt_data_path"] = path.join("joeynmt", "data", src_lang + tgt_lang)

baseline_model_data = "/content/drive/MyDrive/DA/backtranslationhi_en_pmi_nhs_backtranslate_5000/model_data"

os.environ["baseline_model_data"] = baseline_model_data

check_pnt = "/content/drive/MyDrive/DA/model_pmi_nhs_backtranslate_100epochs_new/hien_transformer_pmi_nhs_backtranslate/best.ckpt"

! mkdir -p $jnmt_data_path
! mkdir -p $bpe_data

In [ ]:
!mkdir health_domain_data
!cp $tokenized_data/health_tok.* $tokenized_data/test_tok* /content/health_domain_data

In [ ]:
!wc -l /content/health_domain_data/*

   5464 /content/health_domain_data/health_tok.en
   5464 /content/health_domain_data/health_tok.hi
    598 /content/health_domain_data/test_tok.en
    598 /content/health_domain_data/test_tok.hi
  12124 total


In [ ]:
import pandas as pd

def create_df(src_file, tgt_file):
  src = []
  tgt = []

  with open(src_file, 'r') as src_, open(tgt_file, 'r') as tgt_:
    src_lines = src_.readlines()
    tgt_lines = tgt_.readlines()

  for line_index in range(len(src_lines)):
    src.append(src_lines[line_index].strip('\n'))
    tgt.append(tgt_lines[line_index].strip('\n'))

  df = pd.DataFrame({'src_sentence': src,
                     'tgt_sentence': tgt})

  return df

In [ ]:
df_train_health = create_df("/content/health_domain_data/health_tok.en", "/content/health_domain_data/health_tok.hi" )

In [ ]:
df_train_health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5464 entries, 0 to 5463
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   src_sentence  5464 non-null   object
 1   tgt_sentence  5464 non-null   object
dtypes: object(2)
memory usage: 85.5+ KB


In [ ]:
### splitting data 500 rows from train data for validation purpose
from sklearn.model_selection import train_test_split

df_train, df_dev = train_test_split(df_train_health, test_size=0.092, random_state=42, shuffle=True)

df_dev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 503 entries, 5062 to 2412
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   src_sentence  503 non-null    object
 1   tgt_sentence  503 non-null    object
dtypes: object(2)
memory usage: 11.8+ KB


In [ ]:
def create_tok_files(input_df, dataset_name):
  src_tok_file = dataset_name + "_tok.en"
  tgt_tok_file = dataset_name + "_tok.hi"
  src_content = input_df['src_sentence'].to_list()
  tgt_content = input_df['tgt_sentence'].to_list()

  with open(src_tok_file, 'w') as src, open(tgt_tok_file, 'w') as tgt:
    for src_line, tgt_line in zip(src_content, tgt_content):
      src.write(src_line + "\n")
      tgt.write(tgt_line + "\n")


In [ ]:
create_tok_files(df_train, "train")
create_tok_files(df_dev, "dev")

In [ ]:
!cp /content/health_domain_data/*test* .

In [ ]:
### Learn BPEs on training data
from os import path

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$src_lang < train_tok.$src_lang > train.bpe.$src_lang
! subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$tgt_lang < train_tok.$tgt_lang > train.bpe.$tgt_lang

! subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$src_lang < dev_tok.$src_lang > dev.bpe.$src_lang
! subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$tgt_lang < dev_tok.$tgt_lang > dev.bpe.$tgt_lang


! subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$src_lang < test_tok.$src_lang > test.bpe.$src_lang
! subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$tgt_lang < test_tok.$tgt_lang > test.bpe.$tgt_lang


In [ ]:
!cp *.bpe.* $jnmt_data_path

!cp *.bpe.* $bpe_data

!cp $baseline_model_data/vocab.txt $jnmt_data_path

In [ ]:
model_type = "bpe"
use_cuda = True

# Create the Joey NMT config file

config = """
name: "{name}_transformer"

data:
    src: "{src_lang}"
    trg: "{tgt_lang}"
    train: "data/{name}/train.{model_type}"
    dev:   "data/{name}/dev.{model_type}"
    test:   "data/{name}/test.{model_type}"
    level: "{model_type}"
    lowercase: False
    max_sent_length: 60
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    load_model: "{check_pnt}"
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    learning_rate_warmup: 1000 
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 512
    batch_type: "token"
    eval_batch_size: 300
    eval_batch_type: "token"
    eval_metric: "bleu"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 100            # TODO: Set to at least once per epoch.
    logging_freq: 100
    model_dir: "models/{name}_transformer_{tag}"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: {use_cuda}
    max_output_length: 130
    print_valid_sents: [0, 1, 2, 3]
    keep_best_ckpts: 3
    
model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name = src_lang+tgt_lang, src_lang = src_lang, tgt_lang = tgt_lang, model_type = model_type, use_cuda = use_cuda, tag = tag, check_pnt = check_pnt)
with open("joeynmt/configs/transformer_{name}.yaml".format(name = src_lang+tgt_lang), 'w') as f:
    f.write(config)

In [ ]:
! pip install --upgrade sacrebleu==2.0.0

In [ ]:
# Train the model
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src_lang$tgt_lang.yaml

2021-09-20 11:54:23,981 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-09-20 11:54:24,036 - INFO - joeynmt.data - Loading training data...
2021-09-20 11:54:24,140 - INFO - joeynmt.data - Building vocabulary...
2021-09-20 11:54:24,569 - INFO - joeynmt.data - Loading dev data...
2021-09-20 11:54:24,575 - INFO - joeynmt.data - Loading test data...
2021-09-20 11:54:24,585 - INFO - joeynmt.data - Data loaded.
2021-09-20 11:54:24,585 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-09-20 11:54:24,782 - INFO - joeynmt.model - Enc-dec model built.
2021-09-20 11:54:26,819 - INFO - joeynmt.training - Total params: 12422400
2021-09-20 11:54:31,622 - INFO - joeynmt.training - Loading model from /content/drive/MyDrive/DA/model_pmi_nhs_backtranslate_100epochs_new/hien_transformer_pmi_nhs_backtranslate/best.ckpt
2021-09-20 11:54:34,648 - INFO - joeynmt.helpers - cfg.name                           : hien_transformer
2021-09-20 11:54:34,648 - INFO - joeynmt.helpers - cfg.

In [ ]:
copy_model_to = "/content/drive/MyDrive/DA/model_" + tag + "_30epochs"
!mkdir -p $copy_model_to
os.environ["copy_model_to"] = copy_model_to

!cp -R /content/joeynmt/models/hien_transformer_pmi_nhs_da_bt/ $copy_model_to

In [ ]:
# Output our validation accuracy
! cat "/content/joeynmt/models/hien_transformer_pmi_nhs_da_bt/validations.txt"

Steps: 73100	Loss: 47828.82812	PPL: 14.35309	bleu: 10.32473	LR: 0.00030000	
Steps: 73200	Loss: 44657.67578	PPL: 12.02922	bleu: 11.27906	LR: 0.00030000	
Steps: 73300	Loss: 43523.43359	PPL: 11.29278	bleu: 12.17127	LR: 0.00030000	
Steps: 73400	Loss: 41930.36719	PPL: 10.33394	bleu: 12.08221	LR: 0.00030000	
Steps: 73500	Loss: 41369.22266	PPL: 10.01595	bleu: 12.83952	LR: 0.00030000	
Steps: 73600	Loss: 41136.07422	PPL: 9.88672	bleu: 12.72804	LR: 0.00021000	
Steps: 73700	Loss: 40771.20312	PPL: 9.68783	bleu: 13.57080	LR: 0.00021000	
Steps: 73800	Loss: 39948.00391	PPL: 9.25366	bleu: 14.68682	LR: 0.00021000	
Steps: 73900	Loss: 39494.59375	PPL: 9.02290	bleu: 14.57830	LR: 0.00021000	
Steps: 74000	Loss: 39442.39844	PPL: 8.99671	bleu: 14.34129	LR: 0.00021000	
Steps: 74100	Loss: 39452.15625	PPL: 9.00160	bleu: 14.81507	LR: 0.00021000	
Steps: 74200	Loss: 39005.80078	PPL: 8.78057	bleu: 15.03478	LR: 0.00014700	
Steps: 74300	Loss: 38716.15234	PPL: 8.64005	bleu: 15.30062	LR: 0.00014700	
Steps: 74400	Loss: 3

In [ ]:
# Test our model
! cd joeynmt; python3 -m joeynmt test /content/joeynmt/configs/transformer_hien.yaml

2021-09-20 13:23:10,310 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-09-20 13:23:10,310 - INFO - joeynmt.data - Building vocabulary...
2021-09-20 13:23:10,736 - INFO - joeynmt.data - Loading dev data...
2021-09-20 13:23:10,743 - INFO - joeynmt.data - Loading test data...
2021-09-20 13:23:10,748 - INFO - joeynmt.data - Data loaded.
2021-09-20 13:23:10,763 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 300
2021-09-20 13:23:10,763 - INFO - joeynmt.prediction - Loading model from models/hien_transformer_pmi_nhs_da_bt/87100.ckpt
2021-09-20 13:23:13,720 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-09-20 13:23:13,921 - INFO - joeynmt.model - Enc-dec model built.
2021-09-20 13:23:13,991 - INFO - joeynmt.prediction - Decoding on dev set (data/hien/dev.bpe.en)...
2021-09-20 13:24:00,597 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
2021-09-20 13:24:00,597 - WARNING - sacrebleu - It looks li

In [ ]:
!pip install sacremoses

## clone indic nlp resources repo
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
## install indic nlp library
!pip install indic-nlp-library
## export indic nlp resource library
os.environ["INDIC_RESOURCES_PATH"] = "/content/indic_nlp_resources"

fatal: destination path 'indic_nlp_resources' already exists and is not an empty directory.


In [ ]:
def translate_file(input_file):
  from sacremoses import MosesDetokenizer
  from indicnlp.tokenize import indic_tokenize

  config = "/content/joeynmt/models/hien_transformer_pmi_nhs_da_bt/config.yaml"
  md = MosesTokenizer(lang='en')

  with open(input_file, 'r') as inputf, open('tokenized_file.txt', 'w') as tokfile:
    lines = inputf.readlines()
    
    for line in lines:
      tokenized = indic_tokenize.trivial_tokenize(line.strip())
      tokfile.write(tokenized + "\n")

  !cat tokenized_file.txt | subword-nmt apply-bpe -c $baseline_model_data/$bpe_file --vocabulary $baseline_model_data/vocab.$src_lang > joeynmt/translate.txt
  !cd joeynmt; python3 -m joeynmt translate "$config" < translate.txt > joey_pred.txt
  print()

  with open(input_file, 'r') as inputtext, open('joeynmt/joey_pred.txt', 'r') as trtext:
    input_lines = inputtext.readlines()
    translated_lines = trtext.readlines()

    for line1,line2 in zip(input_lines, translated_lines):
      print("Source Line: ", line1)
      print("Translated Line: ", md(line2))
      print()


In [ ]:
input_data = """It can cause a new continuous cough, fever or loss of, or change in, sense of smell or taste (anosmia).
Generally, corona virus can cause more severe symptoms in people with weakened immune systems, older people and those with long term conditions like diabetes, cancer and chronic lung disease.
This is a rapidly changing situation which is being monitored carefully.
What you can and cannot do depends on your protection level (tier).
Glasgow and Moray are at level 3.
Most of mainland Scotland and some islands are at level 2.
Some islands are at level 1.
Check the protection level of your area
Read the Scottish Governments guidance on protection levels
Get a coronavirus test
Some people are at higher risk of developing severe illness with corona virus.
These people should strictly follow physical distancing measures.
Their household and other contacts should also strictly follow physical distancing advice.
"""

with open('input.txt', 'w') as wf:
  wf.write(input_data)

In [ ]:
translate_file('input.txt')

2021-09-11 15:43:43,849 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-09-11 15:43:44,324 - INFO - joeynmt.prediction - Loading model from models/enhi_transformer_pmi_nhs_da/78200.ckpt
2021-09-11 15:43:48,205 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-09-11 15:43:48,436 - INFO - joeynmt.model - Enc-dec model built.

Source Line:  It can cause a new continuous cough, fever or loss of, or change in, sense of smell or taste (anosmia).

Translated Line:  यह एक नयी निरंतर खाँसी, ज्वर या कमी उत्पन्न कर सकता है, या धूम्रपान की संवेदनशीलता में परिवर्तन कर सकता है।


Source Line:  Generally, corona virus can cause more severe symptoms in people with weakened immune systems, older people and those with long term conditions like diabetes, cancer and chronic lung disease.

Translated Line:  सामान्यतः, हृदय विषाणु उन लोगों में कमजोर प्रतिरक्षी तंत्र, बड़े लोगों और वे मधुमेह, कैंसर और चिरकालिक रोग जैसी दीर्घकालिक परिस्थितियों के साथ अधिक गंभीर लक्षणों का कारण हो 

In [ ]:
!python3 /content/joeynmt/scripts/plot_validations.py /content/joeynmt/models/enhi_transformer_pmi_nhs_da --plot_values bleu PPL --output_path bleu-ppl.png